![example](images/director_shot.jpeg)

# Microsoft Movie Analysis

**Authors:** Paul Waweru Mbugua
***

## Overview

A one-paragraph overview of the project, including the business problem, data, methods, results and recommendations.

## Business Problem

Microsoft, a widely renowned company has been following the movie and film industry, and has been hoping to invest in the industry. Therefore, Microsoft has decided to create a new movie studio. This project aims at exploring the types of films produced in the industry, and their respective perfromance in the box office. An analysis of this performance will provide Microsoft with insights, which will enable them decide on the type of films to create.

***
Therefore, it is critical to explore how other studios' domestics and foreign grosses to gauge the gross amounts Microsoft will likely realize after opening its studios. Also, the highest grossing studios are likely to be Microsoft's main competitors, thus analyzing the type of films they produce can provide a picture of the type of films Microsoft should produce, and the type of films to avoid.

Moreover, analyzing the film grosses over the years can enable Microsoft understand the type of films that are likely to achieve higher sales. 

***
Therefore, there are several questions to consider:
* What are the lowest and highest grossing films over the years?
* What are the highest grossing studios over the years?
* what are the highest grossing genres over the years?
* What are the lowest grossing genres over the years?
* What genres do the highest grossing studios produce over the years?

***

## Data Understanding

This project utilized data from Box Office Mojo and IMDB
***

Box Office Mojo, powered by IMDbPro provides the box office receipts by period and area. 

IMDB contains a searchable database of movies, TV, entertainment programs, and their cast and crew members. They provide box office data, for example, the average rating, run time minutes, language, writers, and other information.  

***

In [1]:
# Import standard packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3

%matplotlib inline

In [2]:
# Loading the Box Office Mojo dataset
bomdf = pd.read_csv('bom.movie_gross.csv')
bomdf.head()


,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [3]:
# Connecting to the IMDB database
conn = sqlite3.connect('im.db')

In [5]:
#Viewing all columns from the movie_basics table in the im database

basics = '''
SELECT *
FROM movie_basics
;
'''
movie_basics = pd.read_sql(basics, conn)
movie_basics.head()

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [4]:
#Viweing all columns from the movie_ratings table in the im database

ratings = '''
SELECT *
FROM movie_ratings
;
'''
movie_ratings = pd.read_sql(ratings, conn)
movie_ratings.head()

,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21


## Data Preparation

Describe and justify the process for preparing the data for analysis.

***
Questions to consider:
* Were there variables you dropped or created?
* How did you address missing values or outliers?
* Why are these choices appropriate given the data and the business problem?
***

In [6]:
# Here you run your code to clean the data

## Data Modeling
Describe and justify the process for analyzing or modeling the data.

***
Questions to consider:
* How did you analyze or model the data?
* How did you iterate on your initial approach to make it better?
* Why are these choices appropriate given the data and the business problem?
***

In [ ]:
# Here you run your code to model the data


## Evaluation
Evaluate how well your work solves the stated business problem.

***
Questions to consider:
* How do you interpret the results?
* How well does your model fit your data? How much better is this than your baseline model?
* How confident are you that your results would generalize beyond the data you have?
* How confident are you that this model would benefit the business if put into use?
***

## Conclusions
Provide your conclusions about the work you've done, including any limitations or next steps.

***
Questions to consider:
* What would you recommend the business do as a result of this work?
* What are some reasons why your analysis might not fully solve the business problem?
* What else could you do in the future to improve this project?
***